In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import glob
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import time
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import gc
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout,BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'sample_submission.csv', 'train']


In [5]:
data = '../input/'
train_files = sorted(glob.glob(data + 'train/*/*'))
# test_path = sorted(glob.glob(data + 'test/*'))

labels = ['HTC-1-M7', 'iPhone-4s', 'iPhone-6', 'LG-Nexus-5x', 'Motorola-Droid-Maxx', 
         'Motorola-Nexus-6', 'Motorola-X', 'Samsung-Galaxy-Note3', 'Samsung-Galaxy-S4',
         'Sony-NEX-7'] 
All_labels = [labels.index(file.split('/')[-2]) for file in train_files] # TO BE CHANGED!!!!!!!

x_train_files, x_test_files, y_train_files, y_test_files = train_test_split(train_files,All_labels,test_size = 0.2, shuffle = True,random_state = 42)

y_train = keras.utils.to_categorical(y_train_files, len(labels)) # one hot vector
y_test = keras.utils.to_categorical(y_test_files, len(labels)) # one hot vector

In [6]:
dim = 256

train_imgs = []
for file in tqdm_notebook(x_train_files):
    img = cv2.imread(file)
    center_x = img.shape[0]//2
    center_y = img.shape[1]//2 

    cropped_img = img[center_x-dim//2:center_x+dim//2, center_y-dim//2:center_y+dim//2, : ]
    
#     cropped_img = cv2.filter2D(cropped_img.astype(np.float32),-1,kernel_filter)
    cropped_img = cropped_img.astype(np.float32)
    
    train_imgs.append(cropped_img)
    del file,img,center_x,center_y,cropped_img
    gc.collect()

train_imgs = np.array(train_imgs)

In [7]:
gc.collect()

0

In [8]:
def myFunc(img):
    kernel_filter = 1/12. * np.array([\
            [-1,  2,  -2,  2, -1],  \
            [ 2, -6,   8, -6,  2],  \
            [-2,  8, -12,  8, -2],  \
            [ 2, -6,   8, -6,  2],  \
            [-1,  2,  -2,  2, -1]])

    pro_img = cv2.filter2D(img,-1,kernel_filter)
    return pro_img

In [9]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip = True,preprocessing_function = myFunc)

datagen.fit(train_imgs)

In [18]:
dim = 256

kernel_filter = 1/12. * np.array([\
            [-1,  2,  -2,  2, -1],  \
            [ 2, -6,   8, -6,  2],  \
            [-2,  8, -12,  8, -2],  \
            [ 2, -6,   8, -6,  2],  \
            [-1,  2,  -2,  2, -1]])


test_imgs = []
for file in tqdm_notebook(x_test_files):
    img = cv2.imread(file)
    center_x = img.shape[0]//2
    center_y = img.shape[1]//2 
    cropped_img = img[center_x-dim//2:center_x+dim//2, center_y-dim//2:center_y+dim//2, : ]
    
    cropped_img = cv2.filter2D(cropped_img.astype(np.float32),-1,kernel_filter)    
    test_imgs.append(cropped_img)
    del file,img,center_x,center_y,cropped_img
    gc.collect()

test_imgs = np.array(test_imgs)

In [29]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 0.0001
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

In [30]:
model=Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2), input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2) ))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(10))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=lr_schedule(0), decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

Learning rate:  0.0001
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 127, 127, 64)      1792      
_________________________________________________________________
batch_normalization_10 (Batc (None, 127, 127, 64)      256       
_________________________________________________________________
activation_19 (Activation)   (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 63, 63, 64)        36928     
_________________________________________________________________
batch_normalization_11 (Batc (None, 63, 63, 64)        256       
_________________________________________________________________
activation_20 (Activation)   (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 61, 61, 32)      

In [31]:
model.fit_generator(datagen.flow(train_imgs, y_train, batch_size=32),steps_per_epoch= 2200 / 32, epochs=250,validation_data=(test_imgs,y_test),callbacks=callbacks) 

Epoch 1/250
Learning rate:  0.0001
69/68 [==============================] - 11s 153ms/step - loss: 2.5587 - acc: 0.1440 - val_loss: 2.1651 - val_acc: 0.1873
Epoch 2/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 2.2247 - acc: 0.1567 - val_loss: 2.1635 - val_acc: 0.1636
Epoch 3/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 2.1580 - acc: 0.1898 - val_loss: 2.0855 - val_acc: 0.2109
Epoch 4/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 2.0157 - acc: 0.2328 - val_loss: 1.8720 - val_acc: 0.2455
Epoch 5/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 1.9005 - acc: 0.2723 - val_loss: 1.7892 - val_acc: 0.3200
Epoch 6/250
Learning rate:  0.0001
69/68 [==============================] - 8s 121ms/step - loss: 1.8297 - acc: 0.3009 - val_loss: 1.6874 - val_acc: 0.3509
Epoch 7/250
Learning rate:  0.0001
69/68 [=====================

69/68 [==============================] - 8s 119ms/step - loss: 0.3691 - acc: 0.8780 - val_loss: 0.8040 - val_acc: 0.7509
Epoch 106/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 0.3841 - acc: 0.8745 - val_loss: 0.7300 - val_acc: 0.7600
Epoch 107/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 0.3660 - acc: 0.8762 - val_loss: 1.2431 - val_acc: 0.7055
Epoch 108/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 0.3657 - acc: 0.8736 - val_loss: 0.8302 - val_acc: 0.7764
Epoch 109/250
Learning rate:  0.0001
69/68 [==============================] - 8s 120ms/step - loss: 0.3794 - acc: 0.8738 - val_loss: 0.7657 - val_acc: 0.7745
Epoch 110/250
Learning rate:  0.0001
69/68 [==============================] - 8s 119ms/step - loss: 0.3666 - acc: 0.8759 - val_loss: 1.7590 - val_acc: 0.6309
Epoch 111/250
Learning rate:  0.0001
69/68 [==============================] - 8s 121ms/st

KeyboardInterrupt: 

In [13]:
# del train_imgs
gc.collect()

216

In [14]:
# dim = 256

# kernel_filter = 1/12. * np.array([\
#             [-1,  2,  -2,  2, -1],  \
#             [ 2, -6,   8, -6,  2],  \
#             [-2,  8, -12,  8, -2],  \
#             [ 2, -6,   8, -6,  2],  \
#             [-1,  2,  -2,  2, -1]])


# test_imgs = []
# for file in tqdm_notebook(x_test_files):
#     img = cv2.imread(file)
#     center_x = img.shape[0]//2
#     center_y = img.shape[1]//2 
#     cropped_img = img[center_x-dim//2:center_x+dim//2, center_y-dim//2:center_y+dim//2, : ]
    
#     cropped_img = cv2.filter2D(cropped_img.astype(np.float32),-1,kernel_filter)    
#     test_imgs.append(cropped_img)
#     del file,img,center_x,center_y,cropped_img
#     gc.collect()

# test_imgs = np.array(test_imgs)

In [28]:
scores = model.evaluate(test_imgs,y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

550/550 [==============================] - 1s 2ms/step
Test loss: 0.5598801576007496
Test accuracy: 0.8199999997832559


In [16]:
# del test_imgs
gc.collect()

14